In [1]:
import os
from pathlib import Path

cwd = Path.cwd()
if cwd.name == "notebooks":
    os.chdir(cwd.parent)

#### Load Dataset

In [2]:
from EmoBox.EmoBox import EmoDataset, EmoEval

In [3]:
model_name = "audio-flamingo-3" # qwen2-audio-instruct, audio-flamingo-3, voxtral-mini
dataset = "iemocap"
fold = 1  # different datasets have different number of folds, which can be find in data/
user_data_dir = "./"
meta_data_dir = "EmoBox/data/"
audio_format = "base64" if model_name in ["voxtral-mini"] else "array"

train = EmoDataset(dataset, user_data_dir, meta_data_dir, fold=fold, split="train", audio_format=audio_format)
test = EmoDataset(dataset, user_data_dir, meta_data_dir, fold=fold, split="test", audio_format=audio_format)

since there is no official valid data, use random split for train valid split, with a ratio of [80, 20]
load in 4446 samples, only 4446 exists in data dir EmoBox/data/
load in 1085 samples, only 1085 exists in data dir EmoBox/data/
Num. training samples 4446
Num. valid samples 0
Num. test samples 1085
Using label_map {'neu': 'Neutral', 'hap': 'Happy', 'ang': 'Angry', 'sad': 'Sad', 'exc': 'Happy'}
since there is no official valid data, use random split for train valid split, with a ratio of [80, 20]
load in 4446 samples, only 4446 exists in data dir EmoBox/data/
load in 1085 samples, only 1085 exists in data dir EmoBox/data/
Num. training samples 4446
Num. valid samples 0
Num. test samples 1085
Using label_map {'neu': 'Neutral', 'hap': 'Happy', 'ang': 'Angry', 'sad': 'Sad', 'exc': 'Happy'}


In [4]:
sample = test[0]
sample

{'key': 'iemocap-Ses01F_impro04_F000',
 'audio': array([ 0.00228882,  0.00183105,  0.00180054, ..., -0.00778198,
        -0.00982666, -0.01132202], shape=(70312,), dtype=float32),
 'label': 'Neutral',
 'gender': 'Female',
 'language': 'English'}

In [5]:
test.label_map.values()

dict_values(['Neutral', 'Happy', 'Angry', 'Sad', 'Happy'])

In [6]:
from collections import Counter
labels =  [data['label'] for data in test]
Counter(labels)

Counter({'Neutral': 384, 'Happy': 278, 'Angry': 229, 'Sad': 194})

#### Load Model

In [7]:
import torch
from mllm_emotion_classifier.models import ModelFactory

device = "cuda" if torch.cuda.is_available() else "cpu"
model = ModelFactory.create(
    name=model_name,
    do_sample=True,
    temperature=0.0001,
    prompt_name="user_labels",
    class_labels=set(train.label_map.values()),
    device=device
)

Loading weights:   0%|          | 0/830 [00:00<?, ?it/s]

In [8]:
data_loader = torch.utils.data.DataLoader(
    dataset=test,
    batch_size=4,
    num_workers=4,
    pin_memory=True,
    drop_last=False,
    collate_fn=model.collate_fn
)

In [9]:
batch = next(iter(data_loader))
inputs, _ = batch
# inputs.keys()

Keyword argument `tokenize` is not a valid argument for this processor and will be ignored.
Keyword argument `return_dict` is not a valid argument for this processor and will be ignored.
Keyword argument `tokenize` is not a valid argument for this processor and will be ignored.
Keyword argument `return_dict` is not a valid argument for this processor and will be ignored.
Keyword argument `tokenize` is not a valid argument for this processor and will be ignored.
Keyword argument `return_dict` is not a valid argument for this processor and will be ignored.
Keyword argument `tokenize` is not a valid argument for this processor and will be ignored.
Keyword argument `return_dict` is not a valid argument for this processor and will be ignored.


In [10]:
inputs['attention_mask'].shape

torch.Size([4, 164])

In [11]:
# from tqdm import tqdm
# predictions, labels = [], []
# i = 0
# for inputs, lbl in tqdm(data_loader, total=len(data_loader)):
#     inputs = {k: v.to(model.device) for k, v in inputs.items()}
#     preds = model.predict(inputs)
#     predictions.extend(preds)
#     labels.extend(lbl)
#     i += 1
#     if i == 100: break

#### Evaluation

In [12]:
from mllm_emotion_classifier.evaluate import Evaluator

evaluator = Evaluator()
evaluator.evaluate(model, data_loader)


Evaluating audio-flamingo-3 on iemocap


Inference:   0%|                                                                                               | 0/272 [00:00<?, ?it/s]Keyword argument `tokenize` is not a valid argument for this processor and will be ignored.
Keyword argument `return_dict` is not a valid argument for this processor and will be ignored.
Keyword argument `tokenize` is not a valid argument for this processor and will be ignored.
Keyword argument `return_dict` is not a valid argument for this processor and will be ignored.
Keyword argument `tokenize` is not a valid argument for this processor and will be ignored.
Keyword argument `return_dict` is not a valid argument for this processor and will be ignored.
Keyword argument `tokenize` is not a valid argument for this processor and will be ignored.
Keyword argument `return_dict` is not a valid argument for this processor and will be ignored.
Inference:   2%|█▉                                                                                     | 6/272 [00:07

KeyboardInterrupt: 

In [ ]:
evaluator.results['metrics']['global']

{'f1_macro': 0.4915,
 'f1_weighted': 0.6213,
 'accuracy_unweighted': 0.5777,
 'accuracy_weighted': 0.623,
 'precision_macro': 0.6071,
 'precision_weighted': 0.7195,
 'recall_macro': 0.4622,
 'recall_weighted': 0.623}